<h1>Web scraping Wikipedia Paris Olympics 2024 page</h1>

In [ ]:
# import the libraires
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

<h3>Start the scraping by using Beautiful Soup</h3>

In [ ]:
# fetch the webpage
url = "https://en.wikipedia.org/wiki/2024_Summer_Olympics"
# get the response
response = requests.get(url)
print(response)

<h3>Parse the content</h3>

In [ ]:
# parse the html content
soup = BeautifulSoup(response.text, "lxml")
print(soup)

In [ ]:
# extract the tables
table_countries_div = soup.find_all("div", class_="div-col")
# store the countries inside a separate list since there are multiple divs with the class of 
# div-col I used conuntires[1] to find the right one
participants = table_countries_div[1]
print(participants)

In [ ]:
# extract the li from the div
countries = participants.find_all("li")
# print every sinlge country with it's athleats
for country in countries:
    print(country.text)

<h3>Transform the into a panda dataframe and clean the data</h3>

In [ ]:
# create a list for the countries and for the athleats
country_list = []
athleats_list = []

# parse the scraped items into the respective lists
for item in countries:
    # Get the text content of the <li> element
    text = item.text.strip()
    print(f"Raw text: {text}")
    
    # eliminate the (host) text form the France name
    if " (host)" in text:
        text = text.replace(" (host)", "")
        print(f"THE NEW TEXT IS: {text}")
        
    # split the country and the athleats
    country_count, athleate_count = text.split("(")
    #remove the bracket form the athleate_count 
    athleate_count = athleate_count.replace(")", "")

    # append to the respective lists
    country_list.append(country_count)
    athleats_list.append(athleate_count)

# create the dataframe
df = pd.DataFrame(
    {
        "Country": country_list,
        "Athlete": athleats_list
    }
)

In [ ]:
# Set Pandas options to display all rows and columns
pd.set_option('display.max_rows', None)  # Display all rows
# pd.set_option('display.max_columns', None)  # Display all columns
# pd.set_option('display.width', None)  # Auto-detect the display width
# pd.set_option('display.max_colwidth', None)  # Display full content of each cell
df

In [ ]:
# Reset Pandas display options to default
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')
# pd.reset_option('display.width')
# pd.reset_option('display.max_colwidth')

In [ ]:
# Save to a CSV file
df.to_csv("olympic_countries.csv", index=False)

In [ ]:
# convert the Athlete column as integer
df["Athlete"] = df["Athlete"].astype(int)

In [ ]:
pd.set_option('display.max_rows', None)
# sort the df by the athlete
sorted_df = df.sort_values(by="Athlete", ascending=False)
sorted_df

In [ ]:
# change the index of the sorted dataframe without creating a new dataframe
sorted_df.reset_index(drop=True, inplace=True)
sorted_df

<h3>Plot the top 10 nations based on the number of athletes</h3>

In [ ]:
# extract the top 10 nations
top_10 = sorted_df.head(10)

# Set the figure size
plt.figure(figsize=(10, 6)) 
# create the bar plot (h is used to create horizontal lines)
plt.barh(top_10["Country"], top_10["Athlete"], color="red") 

# costumize the plot
plt.xlabel("Number of athletes per nation") # x-label
plt.ylabel("Nation") # y-label
plt.title("Top 10 nations by the number of athletes")

# rotating the label of y axis (used if the labels are too long)
plt.yticks(rotation=45)
# Save the plot as a PNG file
plt.savefig('athletes_plot.png', dpi=300, bbox_inches='tight')

In [ ]:
# Reset Pandas display options to default
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.width')
pd.reset_option('display.max_colwidth')

<h1>Extracting the medal table and plotting the results</h1>
<h4>Find the table</h4>
<p>Extract it using find() and extract the table rows</p>

<h4>Handle the data</h4>
<p>Find the useful information ad put into a panda data frame</p>

In [ ]:
# extract the table
medal_table = soup.find("table", class_="wikitable sortable sticky-header-multi plainrowheaders jquery-tablesorter")
medal_table

In [ ]:
# extract the tr form the table
rows = medal_table.find_all("tr")
rows

In [ ]:
for medal in rows:
    print(medal.text)

In [ ]:
# let's transform the table in a panda data frame

# intialize list to store the data
data = []

# loop through the rows and store the data
for row in rows:
    # extract the all the cells including the headers
    cells = row.find_all(["th", "td"])
    # Extract the text from each cell and strip any extra whitespace
    row_data = [cell.text.strip() for cell in cells]

    # Append the row data to the list
    data.append(row_data)

# Create a Pandas DataFrame
# Use the first row as the column headers
headers = data[0]  # The first row contains the headers
rows = data[1:]    # The remaining rows contain the data

# Create the DataFrame
df2 = pd.DataFrame(rows, columns=headers)

# Display the DataFrame
df2

In [ ]:
df2.info()

In [ ]:
# rename the column and set the rank column as the index
df2.rename(columns={"NOC": "Country"}, inplace=True)
df2.set_index("Rank", inplace=True)
df2

In [ ]:
# Save to a CSV file
df2.to_csv("olympic_medals.csv", index=False)

In [ ]:
df2.info()

In [ ]:
# copy() creates an indipendent dataframe [BEST PRACTICE WHEN USING SLICES OF DATAFRAME]
top_10_rank = df2.head(10).copy()
top_10_rank

In [ ]:
# clean the unwanted characters
top_10_rank["Country"] = top_10_rank["Country"].str.replace(r"[‡*]", "", regex=True)
# Strip extra spaces
top_10_rank['Country'] = top_10_rank['Country'].str.strip()
top_10_rank

In [ ]:
# Convert the columns in the appropriate data type
top_10_rank["Gold"] = top_10_rank["Gold"].astype(int)
top_10_rank["Silver"] = top_10_rank["Silver"].astype(int)
top_10_rank["Bronze"] = top_10_rank["Bronze"].astype(int)
top_10_rank["Total"] = top_10_rank["Total"].astype(int)
top_10_rank.info()

<h3>Plotting the nations with the most medals</h3>

In [ ]:
# create the position for the x label
x = np.arange(len(top_10_rank["Country"]))
# define the width of every bar
width = 0.2

fig, ax = plt.subplots(figsize=(12, 6))

# create separate bars
gold_bars = ax.bar(x - 1.5 * width, top_10_rank['Gold'], width, label='Gold', color='gold')
silver_bars = ax.bar(x - 0.5 * width, top_10_rank['Silver'], width, label='Silver', color='silver')
bronze_bars = ax.bar(x + 0.5 * width, top_10_rank['Bronze'], width, label='Bronze', color='peru')
total_bars = ax.bar(x + 1.5 * width, top_10_rank['Total'], width, label='Total', color='skyblue')

# Function to add labels on top of the bars
def add_labels(bars): # add_labels() This function iterates through each bar in a bar group and adds a text annotation at the top of the bar.
    for bar in bars:
        height = bar.get_height() # Gets the height of the bar (i.e., the number of medals)
        ax.text(bar.get_x() + bar.get_width() / 2, # Calculates the x-position for the text (center of the bar)
                height, f'{height}', 
                ha='center', va='bottom', # Aligns the text horizontally centered and vertically above the bar
                fontsize=9)
# Add labels on top of each bar
add_labels(gold_bars)
add_labels(silver_bars)
add_labels(bronze_bars)
add_labels(total_bars)

# Add labels, title, and legend
ax.set_xlabel('Country')
ax.set_ylabel('Medals')
ax.set_title('Medal Count by Country (Gold, Silver, Bronze, Total)')
ax.set_xticks(x)
ax.set_xticklabels(top_10_rank["Country"], rotation=45, ha='right')
ax.legend()
# Save the plot as a PNG file
plt.savefig('medal_plot.png', dpi=300, bbox_inches='tight')